# 1. Loading and organizing data

In [1]:
# loading datasets
import pandas as pd

# Vertebral Column
# dataset for classification between Normal (NO) and Abnormal (AB)
vc2c = pd.read_csv('vertebral_column_data/column_2C.dat', delim_whitespace=True, header=None)
# dataset for classification between DH (Disk Hernia), Spondylolisthesis (SL) and Normal (NO)
vc3c = pd.read_csv('vertebral_column_data/column_3C.dat', delim_whitespace=True, header=None)

# Wall-Following
# dataset with all 24 ultrassound sensors readings
wf24f = pd.read_csv('wall_following_data/sensor_readings_24.data', header=None)
# dataset with simplified 4 readings (front, left, right and back)
wf4f  = pd.read_csv('wall_following_data/sensor_readings_4.data',  header=None)
# dataset with simplified 2 readings (front and left)
wf2f  = pd.read_csv('wall_following_data/sensor_readings_2.data',  header=None)

# Parkinson (31 people, 23 with Parkinson's disease (PD))
temp = pd.read_csv('parkinson_data/parkinsons.data')
labels = temp.columns.values.tolist()
new_labels = [label for label in labels if label not in ('name')] # taking off column 'name'
pk = temp[new_labels]

In [2]:
pk_features = pk.columns.tolist()
pk_features.remove('status')

# datasets with separation between 'features' and 'labels'
datasets = {
    "vc2c":  {"features": vc2c.iloc[:,0:6],  "labels": pd.get_dummies(vc2c.iloc[:,6],  drop_first=True)},
    "vc3c":  {"features": vc3c.iloc[:,0:6],  "labels": pd.get_dummies(vc3c.iloc[:,6],  drop_first=True)},
    "wf24f": {"features": wf24f.iloc[:,0:24],"labels": pd.get_dummies(wf24f.iloc[:,24],drop_first=True)},
    "wf4f":  {"features": wf4f.iloc[:,0:4],  "labels": pd.get_dummies(wf4f.iloc[:,4],  drop_first=True)},
    "wf2f":  {"features": wf2f.iloc[:,0:2],  "labels": pd.get_dummies(wf2f.iloc[:,2],  drop_first=True)},
    "pk":    {"features": pk.loc[:,pk_features], "labels": pk.loc[:,["status"]]}
}

In [3]:
for dataset in datasets:
    print("Dataset name: {}\nNumber of features: {}\nNumber of samples: {}\n".format(
        dataset, datasets[dataset]["features"].shape[1], datasets[dataset]["features"].shape[0]
    ))

Dataset name: vc2c
Number of features: 6
Number of samples: 310

Dataset name: vc3c
Number of features: 6
Number of samples: 310

Dataset name: wf24f
Number of features: 24
Number of samples: 5456

Dataset name: wf4f
Number of features: 4
Number of samples: 5456

Dataset name: wf2f
Number of features: 2
Number of samples: 5456

Dataset name: pk
Number of features: 22
Number of samples: 195



In [8]:
# Auxiliary function to tell when processing is over
def is_over(): # linux os
    import os
    os.system('spd-say "your program has finished"')

# 2. SOM studies

In [4]:
# importing old som
%run -i 'som.py'

In [200]:
def h_neighbor(idx_1, idx_2, sigma):
    dist_2 = np.sum((idx_2 - idx_1)**2, axis=1) # norm**2
    print(dist_2)
    return np.exp( -dist_2/(2*sigma**2) ).shape
    
ones = np.ones((20,2))
h_neighbor([0,1], ones, 3)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


(20,)

In [183]:

def index1Dto2D(som, index): # convert index of neurons parameter matrix to the 2D grid index
        return np.asarray([np.ceil((index+1)/som.nRows)-1, index%som.nRows])
    
#for i in range(som_new.neurons.shape[0]):
#    print(som_new.index1Dto2D(i))
indexes = [i for i in range(som_new.neurons.shape[0])]
np.asarray(indexes)
index1Dto2D(som_new,np.asarray(indexes))
#index1Dto2D(som_new,np.array([[1],[2]]))

array([[[0.],
        [0.]],

       [[1.],
        [2.]]])

In [461]:
M = np.ones((10,2))
h = np.array([0,1,2,3,4,5,6,7,8,9])
alpha = 2

print(M, M.shape)
print(" ")
print(h, h.shape)
print(" ")
#op += alpha*M*h[:, np.newaxis]

op = alpha* (M.T * h).T
print(op, op.shape)

[[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]] (10, 2)
 
[0 1 2 3 4 5 6 7 8 9] (10,)
 
[[ 0.  0.]
 [ 2.  2.]
 [ 4.  4.]
 [ 6.  6.]
 [ 8.  8.]
 [10. 10.]
 [12. 12.]
 [14. 14.]
 [16. 16.]
 [18. 18.]] (10, 2)


In [363]:
print(X.shape)
print(h_ik_new.shape)

(195, 22)


NameError: name 'h_ik_new' is not defined

In [531]:
# New SOM
import numpy as np
import plotly.offline as plt
import plotly.graph_objs as go
from random import randint
import ipywidgets as widgets
from IPython.display import clear_output
from plotly import tools
from multiprocessing import Pool

plt.init_notebook_mode(connected=True) # enabling plotly inside jupyter notebook

class SOM:
    'Class of Self Organizing Maps conected in a two-dimensional grid.'
    
    def __init__(self, nRows, nColumns): 
        self.nRows    = nRows
        self.nColumns = nColumns
        self.dim = 0   # neurons dimension = features dimension
        self.__epochs = 0 # number of epochs of trained SOM
        
        self.param   = None 
        self.neurons = None
        #self.neurons2 = None
        
        self.paramHist = None
        self.ssdHist   = None
        
    def init(self, X): # giving the data, so we can define maximum and minimum in each dimension
        self.dim = X.shape[1] # number of features
        self.param = np.zeros((self.dim, self.nRows, self.nColumns))
        self.paramHist = None # reset paramHist and ssdHist
        self.ssdHist   = None
                
        # Auxiliary random element
        rand_01 = np.random.rand(self.dim, self.nRows, self.nColumns)
        # find min-max for each dimension:
        minimum = np.amin(X, axis=0)
        maximum = np.amax(X, axis=0)
        for dim in range(self.dim):
            self.param[dim,:,:] = (maximum[dim]-minimum[dim])*rand_01[dim,:,:] + minimum[dim]
        
    
    #def update_neuron(self, args):
    #    row, column, winner_idx, alpha, sigma, i = args
    #    h_ik = self.h_neighbor(winner_idx, [row, column], sigma)
    #    self.param[:,row,column] += alpha * h_ik * (X[i] - self.param[:,row,column])
        
    
    def fit(self, X, alpha0, sigma0, nEpochs=100, saveParam=False, saveSSD=True, tol=1e-6,
              verboses=0, batchSize=np.inf):
        if self.param is None: self.init(X) # if self.init() wasn't run
            
        self.neurons  = self.paramAsMatrix() #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        #self.neurons2 = self.paramAsMatrix() #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        #temp = np.array([k for k in range(self.neurons2.shape[0])])
        temp = np.array([k for k in range(self.neurons.shape[0])])
        indexes = self.index1Dto2D2(temp)
        
        tau1 = nEpochs/sigma0
        tau2 = nEpochs
        SSD_new = self.SSD(X) # initial SSD, from random parameters
        
        if saveParam: 
            self.paramHist = np.zeros((nEpochs+1, self.dim, self.nRows, self.nColumns))
            self.paramHist[0,:,:,:] = self.param # random parameters
        if saveSSD:
            self.ssdHist = np.zeros((nEpochs+1))
            self.ssdHist[0] = SSD_new # initial SSD, from random parameters
        
        sigma = sigma0
        alpha = alpha0
        inertia = np.inf # initial value of inertia
        batchSize = min(len(X), batchSize) # adjusting ill defined batchSize
        for epoch in range(nEpochs):
            # Updating alpha and sigma
            sigma = sigma0*np.exp(-epoch/tau1);
            alpha = alpha0*np.exp(-epoch/tau2);
            
            order = np.random.permutation(len(X)) # shuffling order
            for i in order[:batchSize]: # for each datapoint in the shuflled order until batchSize
                # search for winner neuron
                winner_idx = self.get_winner(X[i])
                
                # neighbor function
                h_ik_new = self.h_neighbor_new(winner_idx,indexes,sigma)
                #temp1 = np.zeros(self.neurons.shape)
                
                #for index in range(self.neurons.shape[0]):
                #    self.neurons[index] += alpha * h_ik_new[index] * (X[i] - self.neurons[index])
                    
                #    temp1[index,:] = alpha * h_ik_new[index] * (X[i] - self.neurons[index])
                
                #temp2 = (h_ik_new[:, np.newaxis]*(X[i] - self.neurons2))*alpha #alpha*h_ik_new*(
                #print(temp2.shape)
                #test = np.all(temp1==temp2)
                #if not test: print(temp1-temp2)
                
                #print(h_ik_new.shape,self.neurons2.shape, alpha)
                #self.neurons2 += (h_ik_new[:, np.newaxis]*(X[i] - self.neurons2))*alpha
                self.neurons += (h_ik_new[:, np.newaxis]*(X[i] - self.neurons))*alpha
                
            
            #print(self.neurons.shape,self.neurons2.shape)
            #test = np.all(self.neurons==self.neurons2) 
            #if not test: print(np.sum(np.absolute(self.neurons-self.neurons2)))
                                    
            self.__epochs = epoch+1 # saving number of epochs
            if verboses==1: print("End of epoch {}".format(self.__epochs))
            
            SSD_old = SSD_new
            SSD_new = self.SSD(X)
            inertia = abs((SSD_old - SSD_new)/SSD_old)
            
            self.param = self.matrixAsParam(self.neurons)
            
            # Saving if necessary
            if saveParam: self.paramHist[epoch+1,:,:,:] = self.matrixAsParam(self.neurons)
            if saveSSD:   self.ssdHist[epoch+1]         = SSD_new
                       
            # breaking if tolerance reached before nEpochs
            if inertia < tol:
                # history cutting
                if saveParam: self.paramHist = self.paramHist[0:epoch+2,:,:,:]
                if saveSSD:   self.ssdHist   = self.ssdHist[0:epoch+2]
                break
            
            
    def SSD(self, X):
        SSD = 0
        for x in X: # can it be vectorized?
            dist_2 = np.sum((self.neurons - x)**2, axis=1) # norm**2
            SSD += np.amin(dist_2)
        return SSD
        
    def index1Dto2D(self, index): # convert index of neurons parameter matrix to the 2D grid index
        return [ceil((index+1)/self.nRows)-1, index%self.nRows]
    
    def index1Dto2D2(self, index): # convert index of neurons parameter matrix to the 2D grid index
        return np.asarray([np.ceil((index+1)/self.nRows)-1, index%self.nRows]).T
    
    def get_winner(self, x):
        dist_2 = np.sum((self.neurons - x)**2, axis=1) # norm**2
        temp = np.argmin(dist_2)
        return self.index1Dto2D(temp)
        
    def h_neighbor(self, idx_1, idx_2, sigma):
        aux = np.asarray(idx_1) - np.asarray(idx_2)
        return np.exp( -np.dot(aux,aux)/(2*sigma**2) )
    
    def h_neighbor_new(self, idx_1, idx_2, sigma):
        dist_2 = np.sum((idx_2 - idx_1)**2, axis=1) # norm**2
        #print(dist_2.shape)
        return np.exp( -dist_2/(2*sigma**2) )
    
    def getLabels(self, X):
        N = len(X)
        labels = np.zeros((N,2))
        #labels = [self.get_winner(X[i,:]) for i in range(len(X))]
        for i in range(N):
            labels[i,:] = self.get_winner(X[i,:])
            
        return labels
    

    def paramAsMatrix(self): # return the 3D matrix of param as a 2D matrix as in k-means
        som_clusters = np.zeros((self.nRows*self.nColumns, self.dim))
        count=0
        for r in range(self.nRows):
            for c in range(self.nColumns):
                som_clusters[count] = self.param[:,r,c]
                count+=1
        return som_clusters
    
    def matrixAsParam(self, matrix):
        param = np.zeros((self.dim, self.nRows, self.nColumns))
        for index in range(matrix.shape[0]):
            [r, c] = self.index1Dto2D(index)
            param[:,r,c] = matrix[index]
        return param
    
    
    def plotSSD(self):
        traceData = go.Scatter(
            x = [i+1 for i in range(self.__epochs)], # epochs
            y = self.ssdHist, 
            mode='lines',
            name='SSD')
        data = [traceData]
        layoutData = go.Layout(
            title = "SSD history",
            xaxis=dict(title='Epoch'),
            yaxis=dict(title='SSD')
        )

        fig = go.Figure(data=data, layout=layoutData)
        plt.iplot(fig)
    
        
    # function to plot SOM in the special case when the feature space is 2D
    def plotSOM(self, X): 
        if self.paramHist is not None:
            # Int box to change the iteration number
            n_txt = widgets.BoundedIntText(
                value=0,
                min=0,
                max=len(self.paramHist)-1,
                step=10,
                description='epoch:'
            )    

        # Function to draw the graph
        def atualizarGrafico(change):
            clear_output()

            if self.paramHist is not None:
                display(n_txt)    
                n_ = change['new'] # new iteration number

            if X is not None:
                datapoints = go.Scatter(
                    x = X[:,0], 
                    y = X[:,1], 
                    mode='markers',
                    name='data',
                    marker = dict(
                         size = 5,
                         color = '#03A9F4'
                        )
                )

                if self.paramHist is not None:
                    x = self.paramHist[n_,0,:,:].reshape(-1).tolist() 
                    y = self.paramHist[n_,1,:,:].reshape(-1).tolist()
                    name = 'neurons [epoch ='+str(n_)+']'
                else:
                    x = self.param[0,:,:].reshape(-1).tolist()
                    y = self.param[1,:,:].reshape(-1).tolist()
                    name = 'neurons'

                neurons = go.Scatter(x=x, y=y, mode='markers', name=name, 
                                     marker = dict(size=10,color = '#673AB7'))

                data = [datapoints, neurons]

                # cada linha que conecta os neurônios
                linhas = [{}]*(2*self.nRows*self.nColumns - self.nRows - self.nColumns)
                count=0 #contador para saber qual linha estamos
                for linha in range(self.nRows): # conecta da esquerda para direita
                    for coluna in range(self.nColumns): # e de cima para baixo
                        try:
                            if self.paramHist is not None:
                                x0 = self.paramHist[n_,0,linha,coluna]
                                y0 = self.paramHist[n_,1,linha,coluna]
                                x1 = self.paramHist[n_,0,linha,coluna+1]
                                y1 = self.paramHist[n_,1,linha,coluna+1]
                            else:
                                x0 = self.param[0,linha,coluna]
                                y0 = self.param[1,linha,coluna]
                                x1 = self.param[0,linha,coluna+1]
                                y1 = self.param[1,linha,coluna+1]

                            linhas[count]= {'type':'line','x0':x0,'y0': y0,'x1':x1,'y1':y1,
                                            'line': {'color': '#673AB7','width': 1,}}
                            count+=1
                        except:
                            pass
                        try:
                            if self.paramHist is not None:
                                x0 = self.paramHist[n_,0,linha,coluna]
                                y0 = self.paramHist[n_,1,linha,coluna]
                                x1 = self.paramHist[n_,0,linha+1,coluna]
                                y1 = self.paramHist[n_,1,linha+1,coluna]
                            else:
                                x0 = self.param[0,linha,coluna]
                                y0 = self.param[1,linha,coluna]
                                x1 = self.param[0,linha+1,coluna]
                                y1 = self.param[1,linha+1,coluna]

                            linhas[count] = {'type': 'line','x0': x0,'y0': y0,'x1': x1,'y1': y1,
                                             'line': {'color': '#673AB7','width': 1}}
                            count+=1
                        except:
                            pass

                layout = go.Layout(
                    title = "Dados + SOM",
                    xaxis=dict(title="$x_1$"),
                    yaxis=dict(title="$x_2$"),
                    shapes=linhas
                )

                fig = go.Figure(data=data, layout=layout)
                plt.iplot(fig)

        if self.paramHist is not None:
            n_txt.observe(atualizarGrafico, names='value')

        atualizarGrafico({'new': 0})

# Testing consistency:

In [538]:
%%time
from math import ceil
# choosing and preparing dataset

dataset_name='pk'

data = datasets[dataset_name]
N = len(data['features'].index) # number of datapoints
l = ceil((5*N**.5)**.5) # tamanho do lado da grid quadrada de neurônios
X = data['features'].values.copy()
X1, X2 = scale_feat(X,X,scaleType='min-max')
X=X1

params = {
     "X":         X
    ,"alpha0":    0.1
    ,"sigma0":    3 
    ,"nEpochs":   1000
    ,"verboses":  0
    ,"saveParam": False
}

som_new = SOM(l, l)
som_new.fit(**params)
som_new.plotSSD()

CPU times: user 9.3 s, sys: 0 ns, total: 9.3 s
Wall time: 9.3 s


In [536]:
%%time
from math import ceil
# choosing and preparing dataset

dataset_name='wf2f'

data = datasets[dataset_name]
N = len(data['features'].index) # number of datapoints
l = ceil((5*N**.5)**.5) # tamanho do lado da grid quadrada de neurônios
X = data['features'].values.copy()
X1, X2 = scale_feat(X,X,scaleType='min-max')
X=X1

params = {
     "X":         X
    ,"alpha0":    0.1
    ,"sigma0":    3 
    ,"nEpochs":   1000
    ,"verboses":  0
    ,"saveParam": True
}

som_new = SOM(l, l)
som_new.fit(**params)
som_new.plotSSD()
# nEpochs = 10
# Wall time: 1min 17s

CPU times: user 5min 16s, sys: 0 ns, total: 5min 16s
Wall time: 5min 16s


In [537]:
som_new.plotSOM(X)
is_over()

BoundedIntText(value=799, description='epoch:', max=799, step=10)

## Performance comparison:

In [539]:
%%time
import time
import datetime
from math import ceil

# choosing and preparing dataset
dataset_name='pk'
data = datasets[dataset_name]
N = len(data['features'].index) # number of datapoints
l = ceil((5*N**.5)**.5) # tamanho do lado da grid quadrada de neurônios
X = data['features'].values.copy()
X1, X2 = scale_feat(X,X,scaleType='min-max')
X=X1

n = 100

params = {
     "X":         X
    ,"alpha0":    0.1
    ,"sigma0":    3 
    ,"nEpochs":   2 
    ,"verboses":  0
    ,"saveParam": False
}

som_old = SOM_2D(l, l, X.shape[1]); som_old.init(X)
som_new = SOM(l, l)

print("n={}".format(n))

t0 = time.time()
for i in range(n): som_old.train(**params, batchSize=X.shape[0])
t1 = time.time()
total_old = (t1-t0)/n
print("som_old finished at {}".format(datetime.datetime.now()))

t0 = time.time()
for i in range(n): som_new.fit(**params)
t1 = time.time()
total_new = (t1-t0)/n
print("som_new finished at {}".format(datetime.datetime.now()))

print("\nOld SOM time: {}\nNew SOM time: {}\n".format(total_old,total_new))
print("New SOM {}x faster.\n".format(total_old/total_new))

n=100
som_old finished at 2019-06-30 01:36:54.546386
som_new finished at 2019-06-30 01:36:57.183403

Old SOM time: 0.7030304741859436
New SOM time: 0.026369287967681884

New SOM 26.660957817578367x faster.

CPU times: user 1min 12s, sys: 23.6 ms, total: 1min 12s
Wall time: 1min 12s
